In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 71kB/s 
     |████████████████████████████████| 204kB 51.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=ae08d1f7386ddad2874b7cbad70b0b96049f2f65966d095fe6b5fa9e626daf51
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
from pyspark import SparkContext 
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.types import *
from google.colab import drive
sc = SparkContext()
sqlContext = SQLContext(sc)
json_file_path = '/content/drive/MyDrive/學業/巨量/期末/final/yelp_academic_dataset_business.json'
#json_file_path = '/content/drive/My Drive/yelp_academic_dataset_business.json'
df = sqlContext.read.json(json_file_path)

In [4]:
print((df.count(), len(df.columns)))
df.printSchema()

(209393, 14)
root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodFo

In [5]:
import pyspark.sql.functions as F
RestaurantsPriceRange2 = df.select(F.col("business_id").alias("business_id"),F.col("attributes.RestaurantsPriceRange2").alias("RestaurantsPriceRange2"))

print((RestaurantsPriceRange2.count(), len(RestaurantsPriceRange2.columns)))
df_concat = df.join(RestaurantsPriceRange2, ['business_id'])
print((df_concat.count(), len(df_concat.columns)))

(209393, 2)
(209393, 15)


1. Chain：如果商店名稱出現超過一次，就算連鎖

In [6]:
df_1=df.groupBy("name").count().where("count >= 1")
df_1 = df_1.withColumn("is_chain", F.when(df_1["count"] > 1, 1).otherwise(0))
df_final=df.select(F.col("business_id").alias("business_id"),F.col("name").alias("name"))
df_final=df_final.join(df_1,['name'])

In [7]:
print((df_final.count(),len(df_final.columns)))
df_final.show()

(209393, 4)
+--------------------+--------------------+-----+--------+
|                name|         business_id|count|is_chain|
+--------------------+--------------------+-----+--------+
|The Range At Lake...|f9NumwFMBDn751xgF...|    1|       0|
|   Carlos Santo, NMD|Yzvjg0SayhoZgCljU...|    1|       0|
|             Felinus|XNoUzKckATkOD1hP6...|    1|       0|
|Nevada House of Hose|6OAZjbxqM5ol29BuH...|    2|       1|
|USE MY GUY SERVIC...|51M2Kk903DFYI6gnB...|    1|       0|
|Oasis Auto Center...|cKyLV5oWZJ2NudWgq...|    1|       0|
|Green World Cleaners|oiAlXZPIFm2nBCt0D...|    1|       0|
|Junction Tire & A...|ScYkbYNkDgCneBrD9...|    1|       0|
| The Empanadas House|pQeaRpvuhoEqudo3u...|    1|       0|
|      Xtreme Couture|EosRKXIGeSWFYWwpk...|    1|       0|
|Chinook Landscapi...|MbZMmwo-eL0Jnm_Yb...|    1|       0|
|Dependable Brakes...|7Dv4_HAxsxvadEsT5...|    1|       0|
|Chocolate Shoppe ...|M_guz7Dj7hX0evS67...|    7|       1|
|        Convertabath|JjJs3o60uQCfctDjs...| 

In [8]:
df_final= df_final.select(F.col("name"),F.col("business_id"),F.col("count").alias("count_chain"),F.col("is_chain"))
df_final.show()

+--------------------+--------------------+-----------+--------+
|                name|         business_id|count_chain|is_chain|
+--------------------+--------------------+-----------+--------+
|The Range At Lake...|f9NumwFMBDn751xgF...|          1|       0|
|   Carlos Santo, NMD|Yzvjg0SayhoZgCljU...|          1|       0|
|             Felinus|XNoUzKckATkOD1hP6...|          1|       0|
|Nevada House of Hose|6OAZjbxqM5ol29BuH...|          2|       1|
|USE MY GUY SERVIC...|51M2Kk903DFYI6gnB...|          1|       0|
|Oasis Auto Center...|cKyLV5oWZJ2NudWgq...|          1|       0|
|Green World Cleaners|oiAlXZPIFm2nBCt0D...|          1|       0|
|Junction Tire & A...|ScYkbYNkDgCneBrD9...|          1|       0|
| The Empanadas House|pQeaRpvuhoEqudo3u...|          1|       0|
|      Xtreme Couture|EosRKXIGeSWFYWwpk...|          1|       0|
|Chinook Landscapi...|MbZMmwo-eL0Jnm_Yb...|          1|       0|
|Dependable Brakes...|7Dv4_HAxsxvadEsT5...|          1|       0|
|Chocolate Shoppe ...|M_g

In [9]:
print((df_final.count(), len(df_final.columns)))

(209393, 4)


2. Similar Restaurant Density: 找一公里內鄰居數量去算密集度

In [10]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import acos, cos, sin, lit
from pyspark.sql import SparkSession
import math

def haversine(lon1, lat1, lon2, lat2):
  lon1, lat1, lon2, lat2 = math.radians(lon1), math.radians(lat1), math.radians(lon2), math.radians(lat2)
  dlon = lon2 - lon1 
  dlat = lat2 - lat1 
  a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
  c = 2 * math.atan2( math.sqrt(a), math.sqrt(1-a)) 
  r = 6371 # Radius of earth in kilometers
  return c * r

myudf= udf(haversine, DoubleType())
#df.createOrReplaceTempView("df_temp")
#spark = SparkSession.builder.appName('Dataframe example').getOrCreate()
#df_coor = spark.sql('select business_id,latitude,longitude from df_temp')
from pyspark.sql.functions import col

In [11]:
df_dist_x = df.select(col("business_id").alias("x_business"),col("latitude").alias("x_latitude"),col("longitude").alias("x_longitude"))
df_dist_y = df.select(col("business_id").alias("y_business"),col("latitude").alias("y_latitude"),col("longitude").alias("y_longitude"))

In [12]:
print(sc.version)
cross = df_dist_x.crossJoin(df_dist_y)

3.0.1


In [13]:
cross_dist = cross.withColumn('dist', myudf(col("x_longitude"),col("x_latitude"),col("y_longitude"),col("y_latitude")))

In [14]:
cross_dist_1 = cross_dist.where("dist<1")
cross_dist_1 = cross_dist_1.where("x_business!=y_business")
cross_dist_1 = cross_dist_1.select(col("x_business"),col("y_business"))

In [15]:
df_neighbor_spark = cross_dist_1
df_neighbor_spark.show()

+--------------------+--------------------+
|          x_business|          y_business|
+--------------------+--------------------+
|f9NumwFMBDn751xgF...|RVyB6fnc8na9njyEL...|
|f9NumwFMBDn751xgF...|ZrOx8EgCvy_GDdq6n...|
|f9NumwFMBDn751xgF...|PwXcBUO_51TVBCmSF...|
|f9NumwFMBDn751xgF...|LglX92sujhfzVcvzL...|
|f9NumwFMBDn751xgF...|QNcmzMJO3JsQzQZPS...|
|f9NumwFMBDn751xgF...|eSmQRcvlNzq4lECtU...|
|f9NumwFMBDn751xgF...|xRrHTt8LcWsGgZ7yQ...|
|f9NumwFMBDn751xgF...|5pl0_FxfWHm6FhqII...|
|f9NumwFMBDn751xgF...|e6Ad2Yl9-HANFfLRt...|
|f9NumwFMBDn751xgF...|9cNA7m4DYm-fzmAOQ...|
|f9NumwFMBDn751xgF...|bAsG8WoegLyNZU5QA...|
|f9NumwFMBDn751xgF...|3UPfd9cmG4OiP4j3L...|
|f9NumwFMBDn751xgF...|4uxzhHXP46910bp0u...|
|f9NumwFMBDn751xgF...|tXAw6-Rs_E-ylrQg8...|
|f9NumwFMBDn751xgF...|RpBpV30TruRcQqo5c...|
|f9NumwFMBDn751xgF...|5YyMd29_o8JIh00mR...|
|f9NumwFMBDn751xgF...|Ajw9oYhxVBWn0n4bm...|
|f9NumwFMBDn751xgF...|T7wNyVif2kjtcImmp...|
|f9NumwFMBDn751xgF...|qWh67DZeEBCKCCYVZ...|
|Yzvjg0SayhoZgCljU...|4oQVtzIZgF

In [16]:
df_cat = df.select(col("business_id"),col("categories"))

In [17]:
from pyspark.sql.functions import col, split 
df_cat = df_cat.select(F.col("business_id").alias("business_id"),F.split('categories', ', ').alias('categories'))
df_cat.select("categories").rdd.flatMap(list).collect()

[['Active Life', 'Gun/Rifle Ranges', 'Guns & Ammo', 'Shopping'],
 ['Health & Medical',
  'Fitness & Instruction',
  'Yoga',
  'Active Life',
  'Pilates'],
 ['Pets', 'Pet Services', 'Pet Groomers'],
 ['Hardware Stores',
  'Home Services',
  'Building Supplies',
  'Home & Garden',
  'Shopping'],
 ['Home Services', 'Plumbing', 'Electricians', 'Handyman', 'Contractors'],
 ['Auto Repair', 'Automotive', 'Oil Change Stations', 'Transmission Repair'],
 ['Dry Cleaning & Laundry', 'Local Services', 'Laundry Services'],
 ['Auto Repair', 'Oil Change Stations', 'Automotive', 'Tires'],
 ['Ethnic Food',
  'Food Trucks',
  'Specialty Food',
  'Imported Food',
  'Argentine',
  'Food',
  'Restaurants',
  'Empanadas'],
 ['Martial Arts', 'Gyms', 'Fitness & Instruction', 'Active Life'],
 ['Contractors', 'Landscaping', 'Home Services'],
 ['Automotive', 'Auto Repair'],
 ['Desserts', 'Food', 'Ice Cream & Frozen Yogurt'],
 ['Contractors', 'Home Services', 'Local Services'],
 ['Beauty & Spas', 'Tanning'],
 ['Lo

In [18]:
df_cat_pd = df_cat.select("*").toPandas()

In [19]:
from pyspark.sql.functions import arrays_overlap, array
'''python寫法
df_neighbor_new = df_neighbor
for i in range(len(df_neighbor_new)):
  print(i)
  x_cat = df_cat_pd.loc[df_cat_pd['business_id'] == df_neighbor.loc[i,'x_business'],'categories'].iloc[0]
  y_cat = df_cat_pd.loc[df_cat_pd['business_id'] == df_neighbor.loc[i,'y_business'],'categories'].iloc[0]
  if x_cat is not None and y_cat is not None:
    print(len(list(set(x_cat) & set(y_cat))))
    if len(list(set(x_cat) & set(y_cat))) == 0:
      df_neighbor=df_neighbor.drop(i)
'''
#用pyspark寫寫看
df_neighbor_spark = df_neighbor_spark.alias("a").join(df_cat.alias("b"), df_neighbor_spark['x_business'] == df_cat['business_id']).select("a.x_business", "a.y_business", "b.categories")
#df_neighbor_spark = df_neighbor_spark.alias("a").join(df_cat.alias("b"), df_neighbor_spark['y_business'] == df_cat['business_id']).select("a.x_business", "a.y_business", "b.categories","b.categories")
df_neighbor_spark.show()

+--------------------+--------------------+--------------------+
|          x_business|          y_business|          categories|
+--------------------+--------------------+--------------------+
|f9NumwFMBDn751xgF...|RVyB6fnc8na9njyEL...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|ZrOx8EgCvy_GDdq6n...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|PwXcBUO_51TVBCmSF...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|LglX92sujhfzVcvzL...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|QNcmzMJO3JsQzQZPS...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|eSmQRcvlNzq4lECtU...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|xRrHTt8LcWsGgZ7yQ...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|5pl0_FxfWHm6FhqII...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|e6Ad2Yl9-HANFfLRt...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|9cNA7m4DYm-fzmAOQ...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|bAsG8WoegLyNZU5QA...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|3UPfd9cmG4OiP4j3L...|[Active Life, Gun...|
|f9NumwFMBDn751xgF...|4ux

In [20]:
df_neighbor_spark = df_neighbor_spark.select(col("x_business"),col("y_business"),col("categories").alias("x_categories"))
df_neighbor_spark = df_neighbor_spark.alias("a").join(df_cat.alias("b"), df_neighbor_spark['y_business'] == df_cat['business_id']).select("a.x_business", "a.y_business", "a.x_categories","b.categories")
df_neighbor_spark = df_neighbor_spark.select(col("x_business"),col("y_business"),col("x_categories"),col("categories").alias("y_categories"))
df_neighbor_spark.show()

+--------------------+--------------------+--------------------+--------------------+
|          x_business|          y_business|        x_categories|        y_categories|
+--------------------+--------------------+--------------------+--------------------+
|f9NumwFMBDn751xgF...|RVyB6fnc8na9njyEL...|[Active Life, Gun...|[Boat Repair, Pro...|
|f9NumwFMBDn751xgF...|ZrOx8EgCvy_GDdq6n...|[Active Life, Gun...|     [Home Services]|
|f9NumwFMBDn751xgF...|PwXcBUO_51TVBCmSF...|[Active Life, Gun...|[Food, Nightlife,...|
|f9NumwFMBDn751xgF...|LglX92sujhfzVcvzL...|[Active Life, Gun...|[Carpet Cleaning,...|
|f9NumwFMBDn751xgF...|QNcmzMJO3JsQzQZPS...|[Active Life, Gun...|[Pet Boarding, Pe...|
|f9NumwFMBDn751xgF...|eSmQRcvlNzq4lECtU...|[Active Life, Gun...|[Shutters, Home S...|
|f9NumwFMBDn751xgF...|xRrHTt8LcWsGgZ7yQ...|[Active Life, Gun...|[Bars, Nightlife,...|
|f9NumwFMBDn751xgF...|5pl0_FxfWHm6FhqII...|[Active Life, Gun...|[Home Services, C...|
|f9NumwFMBDn751xgF...|e6Ad2Yl9-HANFfLRt...|[Active Lif

In [21]:
df_neighbor_spark_overlap = df_neighbor_spark.withColumn("cat_match", arrays_overlap(df_neighbor_spark.x_categories,df_neighbor_spark.y_categories))
from pyspark.sql.functions import array_intersect
df_neighbor_spark_intersec = df_neighbor_spark.withColumn('match_list',array_intersect(F.col('x_categories'), F.col('y_categories')))
#df_neighbor_spark = df_neighbor_spark.filter((df_neighbor_spark.cat_match != 'false'))
df_neighbor_spark_intersec = df_neighbor_spark_intersec.withColumn('matched_count', F.size(F.col('match_list')))
df_neighbor_spark_intersec = df_neighbor_spark_intersec.filter((df_neighbor_spark_intersec.matched_count >= 3))
df_neighbor_spark_intersec.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|          x_business|          y_business|        x_categories|        y_categories|          match_list|matched_count|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|Yzvjg0SayhoZgCljU...|ednUf9hnKsJjXMtgu...|[Health & Medical...|[Tai Chi, Yoga, A...|[Fitness & Instru...|            3|
|Yzvjg0SayhoZgCljU...|doogZxl7GbPVrIJAR...|[Health & Medical...|[Physical Therapy...|[Health & Medical...|            3|
|Yzvjg0SayhoZgCljU...|ac_N_B0N75dJCmlvm...|[Health & Medical...|[Pilates, Active ...|[Fitness & Instru...|            3|
|Yzvjg0SayhoZgCljU...|h0un8wQu1vhOITiXG...|[Health & Medical...|[Health & Medical...|[Health & Medical...|            3|
|6OAZjbxqM5ol29BuH...|4a66cRATa2zu5HHb7...|[Hardware Stores,...|[Medical Supplies...|[Hardware Stores,...|            3|
|51M2Kk903DFYI6gnB...|6jeJih_Ufy

In [ ]:
df_2 = df_neighbor_spark_intersec.groupby("x_business").count()
df_2.show()
# UDF for converting column type from vector to double type
#unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

3. Relative Review Count：把相關的商店取REVIEW COUNT加總

In [23]:
relative_review_count = df_neighbor_spark.alias("a").join(df.alias("b"), df_neighbor_spark['y_business'] == df['business_id']).select("a.x_business", "a.y_business", "b.review_count")
#relative_review_count.show()
relative_review_count = relative_review_count.groupBy("x_business").agg(F.col("x_business"), F.sum("review_count"))
relative_review_count.show()

Py4JJavaError: ignored

In [ ]:
df_2 = df_2.alias("a").join(relative_review_count.alias("b"), relative_review_count['x_business'] == df_2['x_business']).select("a.x_business", "a.count", "sum(review_count)")
df_2.show()
# UDF for converting column type from vector to double type
"""
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

# Iterating over columns to be scaled
for i in ["sum(review_count)"]:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")
    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")
    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline = Pipeline(stages=[assembler, scaler])
    # Fitting pipeline on dataframe
    df_2 = pipeline.fit(df_2).transform(df_2).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")
#count normalization
df_2.show(5)
"""

'\nunlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())\n\n# Iterating over columns to be scaled\nfor i in ["sum(review_count)"]:\n    # VectorAssembler Transformation - Converting column to vector type\n    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")\n    # MinMaxScaler Transformation\n    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")\n    # Pipeline of VectorAssembler and MinMaxScaler\n    pipeline = Pipeline(stages=[assembler, scaler])\n    # Fitting pipeline on dataframe\n    df_2 = pipeline.fit(df_2).transform(df_2).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")\n#count normalization\ndf_2.show(5)\n'

4. Restaurant Density: 找一公里內鄰居數量去算密集度(不區分類別)

In [ ]:
restaurant_den = df_neighbor_spark.groupBy("x_business").count()
restaurant_den = restaurant_den.select(col("x_business"),col("count").alias("restaurant_density_count"))
#restaurant_den.show()

In [ ]:
df_2 = df_2.alias("a").join(restaurant_den.alias("b"), restaurant_den['x_business'] == df_2['x_business']).select("a.x_business", "a.count", "a.sum(review_count)","b.restaurant_density_count")
#df_2.show()

In [ ]:
df_2 = df_2.select(col("x_business"),col("count").alias("Similar_store_count"),col("sum(review_count)"),col("restaurant_density_count").alias("store_density_count"))
#df_2.show()

5. Price：直接取RestaurantsPriceRange2欄位

In [ ]:
df_final = df_final.alias("a").join(RestaurantsPriceRange2.alias("b"), RestaurantsPriceRange2['business_id'] == df_final['business_id']).select("a.name", "a.business_id", "a.count_chain","a.is_chain","b.RestaurantsPriceRange2")
#df_final.show()

6. Review Count：直接取review_count欄位
7. Star Rating：直接取stars欄位
8. is_open

In [ ]:
df_final = df_final.select(col("name"),col("business_id"),col("count_chain"),col("is_chain"),col("RestaurantsPriceRange2"))
df_final = df_final.alias("a").join(df.alias("b"), df['business_id'] == df_final['business_id']).select("a.name", "a.business_id", "a.count_chain","a.is_chain","a.RestaurantsPriceRange2","b.review_count","b.stars","b.is_open")
#df_final.show()

9. Std of Star Rating

In [ ]:
#df_neighbor_spark_intersec.show()
star_rate = df_neighbor_spark_intersec.alias("a").join(df.alias("b"), df['business_id'] == df_neighbor_spark_intersec['y_business']).select("a.x_business", "a.y_business", "b.stars")
#star_rate.show()

In [ ]:
from pyspark.sql.functions import stddev
temp = df_neighbor_spark_intersec.groupby("x_business").count()
temp = temp.withColumn("y_business", temp["x_business"]).select(col("x_business"),col("y_business"))
temp = temp.alias("a").join(df.alias("b"), df['business_id'] == temp['y_business']).select("a.x_business", "a.y_business", "b.stars")
#temp.show()

In [ ]:
import functools 
def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 
star_rate = unionAll([temp,star_rate])
std_star_rate = star_rate.groupby("x_business").agg(stddev("stars"))
#std_star_rate.show()

In [ ]:
df_2 = df_2.alias("a").join(std_star_rate.alias("b"), std_star_rate['x_business'] == df_2['x_business']).select("a.x_business", "a.Similar_store_count", "a.sum(review_count)","a.store_density_count","b.stddev_samp(stars)")
#df_2.show()

10. Relative Reviews per Month

In [ ]:
#sc1 = SparkContext()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
json_file_path = '/content/drive/MyDrive/學業/巨量/期末/final/yelp_academic_dataset_review.json'
#df_review = sqlContext1.read.json(json_file_path)
df_review = spark.read.json(json_file_path)
#sqlContext1 = SQLContext(sc)
#df_review.show(10, False)

In [ ]:
df_review = df_review.select(col("business_id"),col("date"),col("review_id"),col("stars"),col("useful"),col("user_id"))
from pyspark.sql.functions import to_timestamp,date_format
df_review = df_review.withColumn("date", to_timestamp(col("date").cast("timestamp"), 'yyyyMMddHHmm')).withColumn("year_month", date_format(col("date"), "yyyyMM"))#df_review.show()

In [ ]:
df_review_per_month = df_review.groupBy('business_id','year_month').agg(F.count('review_id').alias('review_counts_per_month'))
df_review_per_month.show(truncate=False)

+----------------------+----------+-----------------------+
|business_id           |year_month|review_counts_per_month|
+----------------------+----------+-----------------------+
|WunR7VclAddvbCnc-97jzg|201710    |16                     |
|UhCSu92r2XVPmhQET1c2Bw|201110    |2                      |
|0jAexNHRHhkQv4zNYqenBA|201407    |8                      |
|bx2KK3t9RHD0FmLYFLE_jg|201005    |1                      |
|Y3vtXQz3zPc5AVyUAsAj5Q|201607    |10                     |
|jqQwHUrZk8M6uPPw3CPoFg|201612    |1                      |
|NEVA0IYbawceL6kz5v5DAw|201608    |1                      |
|9F3N2jWo_kqK9A9pC3z74Q|201809    |12                     |
|Da6eZFThE9xanUAGNpZxAg|201411    |2                      |
|FaCxpKgHPgUO9vqyOny5CA|201505    |1                      |
|4Nj2ktP2NjCbv2aKeH3ZBg|201807    |10                     |
|wd-pzPYNehT6QTsc1qr7pA|201007    |1                      |
|OjnRf8yDGEBCoUDdchSViw|201411    |3                      |
|D6xj8xnc-R7Y2y18isc0-A|201804    |2    

In [ ]:
from pyspark.sql.window import Window
windowSpec = Window.partitionBy("business_id").orderBy("year_month")
prev_review_per_month = df_review_per_month.withColumn("prev_reviews", F.lag("review_counts_per_month").over(windowSpec))

In [ ]:
result = prev_review_per_month.withColumn('diff', (prev_review_per_month['review_counts_per_month'] - prev_review_per_month['prev_reviews'])/prev_review_per_month['prev_reviews'])
#result.show()

In [ ]:
df_review_ratio = result.groupBy('business_id').agg(F.avg(col('diff')).alias('ratio'))
#df_review_ratio.show()

In [ ]:
df_neighbor_spark_intersec.show()
relative_review_mon = df_neighbor_spark_intersec.alias("a").join(df_review_ratio.alias("b"), df_neighbor_spark_intersec['y_business'] == df_review_ratio['business_id']).select("a.x_business", "a.y_business", "b.ratio")
#relative_review_mon.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|          x_business|          y_business|        x_categories|        y_categories|          match_list|matched_count|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|Yzvjg0SayhoZgCljU...|ednUf9hnKsJjXMtgu...|[Health & Medical...|[Tai Chi, Yoga, A...|[Fitness & Instru...|            3|
|Yzvjg0SayhoZgCljU...|doogZxl7GbPVrIJAR...|[Health & Medical...|[Physical Therapy...|[Health & Medical...|            3|
|Yzvjg0SayhoZgCljU...|ac_N_B0N75dJCmlvm...|[Health & Medical...|[Pilates, Active ...|[Fitness & Instru...|            3|
|Yzvjg0SayhoZgCljU...|h0un8wQu1vhOITiXG...|[Health & Medical...|[Health & Medical...|[Health & Medical...|            3|
|6OAZjbxqM5ol29BuH...|4a66cRATa2zu5HHb7...|[Hardware Stores,...|[Medical Supplies...|[Hardware Stores,...|            3|
|51M2Kk903DFYI6gnB...|6jeJih_Ufy

In [ ]:
relative_review_mon = relative_review_mon.groupBy("x_business").agg(F.avg(col('ratio')).alias('relative_ratio'))
#relative_review_mon.show()

In [ ]:
df_2 = df_2.alias("a").join(relative_review_mon.alias("b"), relative_review_mon['x_business'] == df_2['x_business']).select("a.x_business", "a.Similar_store_count", "a.sum(review_count)","a.store_density_count","a.stddev_samp(stars)","b.relative_ratio")
#df_2.show()

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
final = df_final.alias("a").join(df_2.alias("b"),df_final['business_id']==df_2['x_business']).select("a.name", "a.business_id", "a.count_chain","a.is_chain","a.RestaurantsPriceRange2","a.review_count","a.stars","b.sum(review_count)","b.Similar_store_count","b.store_density_count","b.stddev_samp(stars)","a.is_open", "b.relative_ratio")

#final = df_final.alias("a").join(df_2.alias("b"),df_final['business_id']==df_2['x_business']).select("a.name", "a.business_id", "a.count_chain","a.is_chain","a.RestaurantsPriceRange2","a.review_count","a.stars","b.sum(review_count)","b.Similar_store_count","b.relative_ratio","b.store_density_count","b.stddev_samp(stars)","a.is_open")
#final_pd = final.toPandas()
#final_pd.to_csv('/content/drive/My Drive/final_pre.csv')
#final.repartition(1).write.format('com.databricks.spark.csv').save('/content/drive/My Drive/final_pre.csv',header = 'true')

In [ ]:
final.printSchema()
#final_pd = final.toPandas()
#final_pd.to_csv('/content/drive/MyDrive/碩二上/巨量/final/final_pre.csv')
#final.repartition(1).write.format('com.databricks.spark.csv').save('/content/drive/My Drive/final_pre.csv',header = 'true')

root
 |-- name: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- count_chain: long (nullable = false)
 |-- is_chain: integer (nullable = false)
 |-- RestaurantsPriceRange2: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- stars: double (nullable = true)
 |-- sum(review_count): long (nullable = true)
 |-- Similar_store_count: long (nullable = false)
 |-- store_density_count: long (nullable = false)
 |-- stddev_samp(stars): double (nullable = true)
 |-- is_open: long (nullable = true)
 |-- relative_ratio: double (nullable = true)



In [ ]:
df_2.printSchema()
#df_2 = 

root
 |-- x_business: string (nullable = true)
 |-- Similar_store_count: long (nullable = false)
 |-- sum(review_count): long (nullable = true)
 |-- store_density_count: long (nullable = false)
 |-- stddev_samp(stars): double (nullable = true)
 |-- relative_ratio: double (nullable = true)



In [ ]:
#final_for_csv = final.filter((final.RestaurantsPriceRange2!='NaN'))
#final_for_csv = final_for_csv.filter((final.RestaurantsPriceRange2.isNotNull()))
#df_final.alias("a").join(df_2.alias("b"),df_final['business_id']==df_2['x_business']).select("a.name", "a.business_id", "a.count_chain","a.is_chain","a.RestaurantsPriceRange2","a.review_count","a.stars","b.sum(review_count)","b.Similar_store_count","b.store_density_count","b.stddev_samp(stars)","a.is_open")
#final_for_csv = final_for_csv.drop("")
final_pd = df_2.coalesce(500).write.csv('/content/drive/MyDrive/碩二上/巨量/final/mycsv11.csv')

AnalysisException: ignored

# normalization


In [ ]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
"""
# UDF for converting column type from vector to double type
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

# Iterating over columns to be scaled
for i in ["count_chain"]:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")
    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")
    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline = Pipeline(stages=[assembler, scaler])
    # Fitting pipeline on dataframe
    final = pipeline.fit(final).transform(final).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")
"""

'\n# UDF for converting column type from vector to double type\nunlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())\n\n# Iterating over columns to be scaled\nfor i in ["count_chain"]:\n    # VectorAssembler Transformation - Converting column to vector type\n    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")\n    # MinMaxScaler Transformation\n    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")\n    # Pipeline of VectorAssembler and MinMaxScaler\n    pipeline = Pipeline(stages=[assembler, scaler])\n    # Fitting pipeline on dataframe\n    final = pipeline.fit(final).transform(final).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")\n'

In [ ]:
# = sqlContext.read.csv('/content/drive/MyDrive/碩二上/巨量/final/final_review_output.csv')
final_review_df = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load('/content/drive/MyDrive/學業/巨量/期末/final/final_review_output.csv')


In [ ]:
final_review_df.show(5)

+---+----------+--------------------+-------------+-------------------+-------------------+----+
|_c0|Unnamed: 0|         business_id|review_counts|          sentiment|              ratio| age|
+---+----------+--------------------+-------------+-------------------+-------------------+----+
|  0|         0|ATeOjZzE3VOI-RW7F...|          166|  0.442832817315716|0.12020862615494893|10.2|
|  1|         1|RMjCnixEY5i12Ciqn...|           79| 0.4609072957348457| 0.4571428571428572| 8.4|
|  2|         2|vbUABNAQI2iwN7v0x...|           11|0.27479758477559096|0.07692307692307693| 7.5|
|  3|         3|VHsNB3pdGVcRgs6C3...|          136| 0.4089958842620818|                0.0| 9.2|
|  4|         4|r-W1HPIZ3V6S2DaXh...|           61| 0.8068823324304365|                0.0|10.6|
+---+----------+--------------------+-------------+-------------------+-------------------+----+
only showing top 5 rows



In [ ]:
final_review_df.count()

In [ ]:
final_rv_df = final_review_df.drop('review_counts')
final_rv_df = final_rv_df.drop('_c0')
final_rv_df = final_rv_df.drop('review_counts')
#final_review_df = final_review_df.drop('Unnamed: 0')
final_rv_df.show(5)

+----------+--------------------+-------------------+-------------------+----+
|Unnamed: 0|         business_id|          sentiment|              ratio| age|
+----------+--------------------+-------------------+-------------------+----+
|         0|ATeOjZzE3VOI-RW7F...|  0.442832817315716|0.12020862615494893|10.2|
|         1|RMjCnixEY5i12Ciqn...| 0.4609072957348457| 0.4571428571428572| 8.4|
|         2|vbUABNAQI2iwN7v0x...|0.27479758477559096|0.07692307692307693| 7.5|
|         3|VHsNB3pdGVcRgs6C3...| 0.4089958842620818|                0.0| 9.2|
|         4|r-W1HPIZ3V6S2DaXh...| 0.8068823324304365|                0.0|10.6|
+----------+--------------------+-------------------+-------------------+----+
only showing top 5 rows



In [ ]:
#final_review_df = final_review_df.drop("review_counts")
joined_data = final.join(final_rv_df, final.business_id == final_rv_df.business_id,how='left')
joined_data = joined_data.drop("RestaurantsPriceRange2")

In [ ]:
joined_data.printSchema()

root
 |-- name: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- count_chain: long (nullable = false)
 |-- is_chain: integer (nullable = false)
 |-- review_count: long (nullable = true)
 |-- stars: double (nullable = true)
 |-- sum(review_count): long (nullable = true)
 |-- Similar_store_count: long (nullable = false)
 |-- store_density_count: long (nullable = false)
 |-- stddev_samp(stars): double (nullable = true)
 |-- is_open: long (nullable = true)
 |-- relative_ratio: double (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- business_id: string (nullable = true)
 |-- sentiment: double (nullable = true)
 |-- ratio: double (nullable = true)
 |-- age: double (nullable = true)



In [ ]:
all_data = joined_data.drop("business_id")
all_data = all_data.drop("name")

all_data.printSchema()

root
 |-- count_chain: long (nullable = false)
 |-- is_chain: integer (nullable = false)
 |-- review_count: long (nullable = true)
 |-- stars: double (nullable = true)
 |-- sum(review_count): long (nullable = true)
 |-- Similar_store_count: long (nullable = false)
 |-- store_density_count: long (nullable = false)
 |-- stddev_samp(stars): double (nullable = true)
 |-- is_open: long (nullable = true)
 |-- relative_ratio: double (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- sentiment: double (nullable = true)
 |-- ratio: double (nullable = true)
 |-- age: double (nullable = true)



# subsampling

In [ ]:
sample_data = all_data.sample(False,0.0001,21)

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StringIndexerModel, StandardScaler, Imputer, VectorAssembler, SQLTransformer
spark = SparkSession.builder.appName("Your App").config("spark.sql.broadcastTimeout", "36000").getOrCreate()
assembler = VectorAssembler(inputCols = ['count_chain', 'is_chain', 'review_count', 'stars', 'sum(review_count)', 'Similar_store_count', 'relative_ratio', 'store_density_count', 'stddev_samp(stars)', 'sentiment','ratio','age'], outputCol = "_features")
  
  # Standardize Features
scaler = StandardScaler(inputCol='_features', outputCol='Attributes', withStd=True, withMean=False)


In [ ]:
all_data.count()

KeyboardInterrupt: ignored

In [ ]:
all_data.coalesce(1).write.format('json').save('./all_data.json')

In [ ]:
preprocess = Pipeline(stages = [assembler, scaler]).fit(all_data)

In [ ]:
preprocess.write().overwrite().save("./output/preprocess")

In [ ]:
finalized_data = preprocess.transform(all_data)

In [ ]:
all_data_rdd = spark.createDataFrame(all_data.rdd,schema=all_data.schema)

Py4JJavaError: ignored

In [ ]:
sample_data = sample_data.withColumn("d_count_chain", sample_data["count_chain"].cast(IntegerType()))
sample_data_rdd = spark.createDataFrame(sample_data.rdd,schema=sample_data.schema)
scaler = MinMaxScaler(inputCol="d_count_chain", outputCol="norm_count_chain")
scalerModel = scaler.fit(sample_data_rdd)
scaledData = scalerModel.transform(sample_data_rdd)

Py4JJavaError: ignored

In [ ]:

train_data,test_data = all_data.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.feature import StandardScaler
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['Unnamed: 0', 'count_chain', 'is_chain', 'review_count', 'stars', 'sum(review_count)', 'Similar_store_count', 'relative_ratio', 'store_density_count', 'stddev_samp(stars)', 'sentiment','ratio','age'], outputCol = 'Attributes')
output = assembler.transform(all_data)

#Input vs Output
finalized_data = output.select("Attributes","is_open")


# MinMax


In [ ]:
sample_data_rdd = spark.createDataFrame(sample_data.rdd,schema=sample_data.schema)
scaler = MinMaxScaler(inputCol="Attributes", outputCol="Attributes_")
scalerModel = scaler.fit(sample_data_rdd)
scaledData = scalerModel.transform(sample_data_rdd)
# Standardize Features
#scaler = StandardScaler(inputCol='Attributes',outputCol='Attributes_',withStd=True, withMean=False)
#scaler_data = scaler.fit(finalized_data)
from pyspark.ml.regression import LinearRegression
#Split training and testing data
train_data,test_data = scaler_data.randomSplit([0.8,0.2])

KeyboardInterrupt: ignored

In [ ]:
finalized_data.printSchema()

root
 |-- Attributes: vector (nullable = true)
 |-- is_open: long (nullable = true)



In [ ]:
spark.conf.set("spark.sql.broadcastTimeout", 10000)

In [ ]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

'10485760b'

In [ ]:
columns = ['count_chain', 'review_count', 'stars', 'sum(review_count)', 'Similar_store_count', 'relative_ratio', 'store_density_count', 'stddev_samp(stars)', 'sentiment','ratio','age']
# Standardise each dataframe, column by column
for column in columns:            
    # Standardise the TRAINING data
    train_data = train_data.withColumn(column + '_norm', ((train_data[column] - averages[column])/std_devs[column]))        
    # Standardise the TEST data (using the training mean and std_dev)     
    test_data = test_data.withColumn(column + '_norm', ((test_df[column] - averages[column])/std_devs[column])) 

AnalysisException: ignored

In [ ]:
# Function to normalise (standardise) PySpark dataframes
def standardize_train_test_data(train_df, test_df, columns):
    '''
    Add normalised columns to the input dataframe.
    formula = [(X - mean) / std_dev]
    Inputs : training dataframe, list of column name strings to be normalised
    Returns : dataframe with new normalised columns, averages and std deviation dataframes 
    '''
    # Find the Mean and the Standard Deviation for each column
    aggExpr = []
    aggStd = []
    for column in columns:
        aggExpr.append(F.mean(train_df[column]).alias(column))
        aggStd.append(F.stddev(train_df[column]).alias(column + '_stddev'))
    
    averages = train_df.agg(*aggExpr).collect()[0]
    std_devs = train_df.agg(*aggStd).collect()[0]
    
    # Standardise each dataframe, column by column
    for column in columns:            
        # Standardise the TRAINING data
        train_df = train_df.withColumn(column + '_norm', ((train_df[column] - averages[column])/std_devs[column + '_stddev']))       
    
        # Standardise the TEST data (using the training mean and std_dev)     
        test_df = test_df.withColumn(column + '_norm', ((test_df[column] - averages[column])/std_devs[column + '_stddev']))  
    return train_df, test_df, averages, std_devs
#columns = ['count_chain', 'review_count', 'stars', 'sum(review_count)', 'Similar_store_count', 'relative_ratio', 'store_density_count', 'stddev_samp(stars)', 'sentiment','ratio','age']
columns = ['Attributes']
standardize_train_test_data(train_data, test_data, columns)

In [ ]:

#from pyspark.ml.regression import LinearRegression
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['count_chain', 'is_chain', 'review_count', 'stars', 'sum(review_count)', 'Similar_store_count',  'store_density_count', 'stddev_samp(stars)', 'sentiment','ratio','age'], outputCol = 'Attributes')
output = assembler.transform(all_data)

#Input vs Output
finalized_data = output.select("Attributes","is_open")
#finalized_data_rdd = spark.createDataFrame(finalized_data.rdd,schema=finalized_data.schema)
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])


In [ ]:
train_data_rdd = spark.createDataFrame(train_data.rdd,schema=train_data.schema)
test_data_rdd = spark.createDataFrame(test_data.rdd,schema=test_data.schema)

# XGboost

In [ ]:
from sparkxgb import XGBoostEstimator
from pyspark.sql.functions import monotonicallyIncreasingId 

cut = train_data.count()/2
idxDf = train_data.withColumn("idx", monotonicallyIncreasingId()) 
part1 = train_data.filter(F.col('idx')< cut) 
part2 = train_data.filter()

xgboost = XGBoostEstimator( featuresCol="Attributes", labelCol="is_open", predictionCol="prediction")
for i in range(2):
  training = 
model = xgboost.fit(train_data)

ModuleNotFoundError: ignored

In [ ]:
!pip install sparkxgb

ERROR: Could not find a version that satisfies the requirement sparkxgb (from versions: none)
ERROR: No matching distribution found for sparkxgb


In [ ]:
from spark import XGBoostClassificationModel, XGBoostClassifier
import pandas as pd
import time
# getting the feature names
target = 'is_open'
features = 'Attributes'

# setting parameters. you can find these parameters in the link above.
params = {'eta': 0.1, 'gamma': 0.1, 'missing': 0.0,
          'treeMethod': 'gpu_hist', 'maxDepth': 3, 
          'growPolicy': 'depthwise', 'lambda_': 1.0,
          'subsample': 1.0, 'numRound': 1000,
          'numWorkers': 1, 'verbosity': 1}
          # create XGBoost model object
xgboost = XGBoostClassifier(**params).setLabelCol(target).setFeaturesCols(features)

ModuleNotFoundError: ignored

In [ ]:
!pip install xgboost4j

ERROR: Could not find a version that satisfies the requirement xgboost4j (from versions: none)
ERROR: No matching distribution found for xgboost4j


In [ ]:
# track training time
start_time = time.time()

# train the model
model = xgboost.fit(train_data)

print("GPU Training Time: %s seconds" % (str(time.time() - start_time)))

# LR

In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel,   LogisticRegressionWithSGD
from pyspark import SparkContext
from pyspark.ml.classification import NaiveBayes
lrm = LogisticRegressionWithSGD.train(train_data, iterations=10)
#lr_predict = LogisticRegressionWithSGD.train(train_data_rdd) 

KeyboardInterrupt: ignored

In [ ]:
sqlContext = SQLContext(sc)
regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'is_open',maxIter=10, regParam=0.3, elasticNetParam=0.8)
#Learn to fit the model from training set
#regressor = regressor.fit(train_data)
#regressor.bestModel.write().overwrite().save('/content/drive/MyDrive/碩二上/巨量/final/')
#def predict_for_partition(partition):
  # Load the keras model
#  model = load_model(regressor)
  # Iterate over the rows on the partition and predict
#  for row in partition:
#    prediction = model.predict(row)
#    yield prediction
regressor_result = train_data_rdd.select("*").repartition(2).rdd.mapPartitions(regressor.fit(train_data_rdd))
print("Coefficients: " + str(regressor.coefficients))
print("Intercept: " + str(regressor.intercept))

KeyboardInterrupt: ignored

In [ ]:
print(regressor_result.take(2))

In [ ]:
prediction = regressor.fit(regressor_result)

AttributeError: ignored

In [ ]:
print(sc.defaultParallelism)

2


In [ ]:
#Predict the model
pred.predictions.show()

NameError: ignored

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)